### Looking at SMARCA4 Phos when ARID1A mutated, due to this paper

#### https://pubs.acs.org/doi/10.1021/pr500470h

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import CPTAC

Loading CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [2]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()
trans = CPTAC.get_transcriptomics()

### Effect of ARID1A Mutation on SMARCA4 (BRG1) Phosphorylation

In [19]:
gene = "ARID1A"
protein = "SMARCA4"

p_values = []
sites = []

phosdf = CPTAC.compare_mutations(phos, protein, gene)
phosdf = phosdf.loc[phosdf["Patient_Type"] == "Tumor"].drop("Patient_Type", axis = 1)
#print(phosdf)
for site in phosdf.columns:
    if site is not "Mutation":
        print(site)
        sitedf = CPTAC.compare_mutations(phos, site, gene)
        sitedf = sitedf.loc[sitedf["Patient_Type"] == "Tumor"].drop("Patient_Type", axis=1)
        #print(sitedf)
        mutateddf = sitedf.loc[sitedf["Mutation"]!= "Wildtype"].dropna(axis=0)
        wtdf = sitedf.loc[sitedf["Mutation"]== "Wildtype"].dropna(axis=0)
        #print(mutateddf)
        
        ttest = scipy.stats.ttest_ind(mutateddf[site], wtdf[site])
        sites.append(site)
        p_values.append(ttest[1])
        

        
indexesToRemove=[]

for index in range(0, len(p_values)):
    if np.isnan(p_values[index]):
        indexesToRemove.append(index)

for rem in range( len(indexesToRemove)-1, -1, -1):
    p_values.pop(indexesToRemove[rem])
    sites.pop(indexesToRemove[rem])
        
#print(p_values)
#print(sites)
        

      ZNRF2-S108  ZNRF2-S113  ZNRF2-S116  ZNRF2-S119  ZNRF2-S120  ZNRF2-S135  \
S001         NaN         NaN         NaN         NaN         NaN    0.415000   
S002         NaN    0.179000    0.040000    0.179000         NaN         NaN   
S003         NaN         NaN         NaN         NaN         NaN    0.268000   
S004         NaN   -0.081200   -0.081200         NaN         NaN   -0.368000   
S005         NaN   -0.651000   -0.651000         NaN         NaN    0.000374   
S006      0.3950    0.063850   -0.221300   -0.511000     0.05930    0.441000   
S007         NaN    0.453000    0.473000    0.453000         NaN         NaN   
S008         NaN   -0.920000   -0.786500   -0.596500    -1.11000   -0.256000   
S009         NaN    0.147000    0.147000         NaN         NaN    0.699000   
S010      0.0709   -0.538910   -0.524700    0.020600    -0.00782    1.350000   
S011         NaN         NaN         NaN         NaN         NaN    0.274000   
S012         NaN   -0.020000   -0.177300

### Bonferroni correction

In [17]:
bf_pvals= []
bf_sig_sites = []

pval_cutoff = .05/len(sites)
for i in range(0, len(p_values)):
    if p_values[i] <= pval_cutoff:
        bf_pvals.append(p_values[i])
        bf_sig_sites.append(sites[i])
        
bf_sig_vals = dict(zip(bf_pvals, bf_sig_sites))
print(bf_sig_vals)

{}


### Didn't find anything significant